<a href="https://colab.research.google.com/github/i-p-evstigneev/diplomEvstigneev/blob/main/NeuroEvstigneev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Прогнозирование цены акций Apple с использование Нейросетевых**



In [23]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
filename = ('https://raw.githubusercontent.com/i-p-evstigneev/diplomEvstigneev/main/data/AAPL_000101_220502.csv')
df = pd.read_csv(filename, sep=",")
print(df.shape, df.columns)
df.rename(columns={"<DATE>": "Date", "<TIME>": "Time", "<OPEN>": "Open", "<HIGH>": "High", "<LOW>": "Low", "<CLOSE>": "Close", "<VOL>": "Volume"}, inplace=True)
print(df.shape, df.columns)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y%m%d')
df = df.drop(columns = 'Time')
print(df.shape, df.columns)


In [ ]:
df

In [ ]:
split = 0.85
i_split = int(len(df) * split)
cols = ["Close", "Volume"]
data_train = df.get(cols).values[:i_split]
data_test = df.get(cols).values[i_split:]
len_train = len(data_train)
len_test = len(data_test)
print(len(df), len_train, len_test)

In [26]:
sequence_length = 50; input_dim = 2; batch_size = 32; epochs = 2

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100, input_shape = (sequence_length - 1, input_dim), return_sequences = True),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.LSTM(100, return_sequences = True),
    tf.keras.layers.LSTM(100, return_sequences = False),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(1, activation = 'linear')
])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes = True)

In [30]:
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

In [35]:
def normalise_windows(window_data, single_window=False):
    '''Normalise window with a base value of zero'''
    normalised_data = []
    window_data = [window_data] if single_window else window_data
    for window in window_data:
        normalised_window = []
        for col_i in range(window.shape[1]):
            normalised_col = [((float(p) / float(window[0, col_i])) - 1) for p in window[:, col_i]]
            normalised_window.append(normalised_col)
        normalised_window = np.array(normalised_window).T # reshape and transpose array back into original multidimensional format
        normalised_data.append(normalised_window)
    return np.array(normalised_data)

In [36]:
def _next_window(i, seq_len, normalise):
    '''Generates the next data window from the given index location i'''
    window = data_train[i:i+seq_len]
    window = normalise_windows(window, single_window=True)[0] if normalise else window
    x = window[:-1]
    y = window[-1, [0]]
    return x, y

In [37]:
def get_train_data(seq_len, normalise):
    '''
    Create x, y train data windows
    Warning: batch method, not generative, make sure you have enough memory to
    load data, otherwise use generate_training_window() method.
    '''
    data_x = []
    data_y = []
    for i in range(len_train - seq_len + 1):
        x, y = _next_window(i, seq_len, normalise)
        data_x.append(x)
        data_y.append(y)
    return np.array(data_x), np.array(data_y)


In [38]:
x, y = get_train_data(seq_len = sequence_length, normalise = True)

In [ ]:
print(x, y, x.shape, y.shape)